In [1]:
import igraph as ig
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point, LineString, MultiLineString
import os
import osmnx as ox
import random
import numpy as np
import itertools
import json
import pandas as pd

In [2]:
roads_file_path = "C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\euro-global-map-shp\\euro-global-map-shp\\euro-global-map-shp\\RoadL.shp"
cities_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\euro-global-map-shp\\euro-global-map-shp\\euro-global-map-shp\\BuiltupP.shp"
tunnels_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\OSM\\osm_tunnels_cantabria.shp"


rail_file_path = "C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\euro-regional-map-shp\\FullEurope\\data\\RailrdL.shp"
stops_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\euro-regional-map-shp\\FullEurope\\data\\RailrdC.shp"


energy_lines_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\OSM\\osm_power_lines_cantabria.shp"
generation_points_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\Combined\\combined_osm_power_cantabria_global_power.shp"
substations_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\OSM\\osm_power_subest_cantabria.shp"


networks_dic = {
                'Roads network':{'lines file paths':{'roads':roads_file_path},
                                 'nodes file paths':{'cities':cities_file_path},
                                 'buffer distance':0.01, 'buffer option':'to lines'},
                'Railway network':{'lines file paths':{'train tracks':rail_file_path},
                                   'nodes file paths':{'stations':stops_file_path},
                                   'buffer distance':None,'buffer option':None},
                'Energy network':{'lines file paths':{'lines':energy_lines_file_path},
                                  'nodes file paths':{'power sources':generation_points_file_path,'substations':substations_file_path},
                                  'buffer distance':0.005,'buffer option':'to nodes'}
                }

interconections_dic={
                     'Roads network - Railway network':{'connected elements':['all','all'],'method':'all in buffer distance','buffer distance':0.015},
                     'Railway network - Energy network':{'connected elements':['all','substations'],'method':'closest'},
                     'Roads network - Energy network':{'connected elements':['tunnel','substations'],'method':'closest'},
                     }

################
# En la versión final, habrá ver como meter sin simular usuarios y flujos en nodos y aristas de las redes y las interconexiones
################

gdf_cut = ox.geocode_to_gdf("Cantabria, Spain")


In [3]:
def combine_gdfs(gdf_cut,paths_dic):
    """
    Une varios shapefiles en un solo GeoDataFrame:
    - reproyectados al CRS del ref_gdf
    - recortados (clip) al ref_gdf
    - con columna extra 'type' = clave del diccionario
    - solo mantiene 'type' + geometry
    """
    gdfs = []

    for key, filepath in paths_dic.items():
        # 1. Leer shapefile
        gdf = gpd.read_file(filepath)

        # 2. Reproyectar al CRS de referencia
        gdf = gdf.to_crs(gdf_cut.crs)

        # 3. Clip con el gdf de referencia
        gdf = gpd.clip(gdf, gdf_cut)

        # 4. Añadir columna con la clave
        gdf["type"] = key

        # 5. Mantener solo la columna 'source' + geometry
        gdf = gdf[["type", "geometry"]]

        gdfs.append(gdf)

    # 6. Concatenar todos en un único GeoDataFrame
    result = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdf_cut.crs)

    return result

In [4]:
for network,dic in networks_dic.items():
    networks_dic[network]['lines gdf']=combine_gdfs(gdf_cut,dic['lines file paths'])
    networks_dic[network]['nodes gdf']=combine_gdfs(gdf_cut,dic['nodes file paths'])

In [5]:
def add_tunnels(lines_gdf,tunnels_path_file):
    import warnings
    warnings.filterwarnings("ignore", message=".*Geometry is in a geographic CRS.*")

    # 1️⃣ Leer túneles y re-proyectar
    tunnels_gdf = gpd.read_file(tunnels_path_file)
    tunnels_gdf = tunnels_gdf.to_crs(lines_gdf.crs)

    # 2️⃣ Preparar carreteras y asegurar columna 'tunnel'
    gdf_roads = lines_gdf.copy()
    if 'tunnel' not in gdf_roads.columns:
        gdf_roads['tunnel'] = 'no'

    # 3️⃣ Crear buffer para los túneles
    buffer_distance = 0.001  # ajustar según CRS
    gdf_tunnels_buffer = tunnels_gdf.copy()
    gdf_tunnels_buffer['geometry'] = gdf_tunnels_buffer.geometry.buffer(buffer_distance)

    # 4️⃣ Spatial join
    intersects = gpd.sjoin(
        gdf_roads,
        gdf_tunnels_buffer,
        how="left",
        predicate="intersects"
    )

    # 5️⃣ Índices de carreteras que intersectan túneles
    roads_with_tunnel = intersects[~intersects['index_right'].isna()].index

    # 6️⃣ Marcar la columna 'tunnel'
    lines_gdf['tunnel'] = 'no'  # inicializar
    lines_gdf.loc[roads_with_tunnel, 'tunnel'] = 'yes'

add_tunnels(networks_dic['Roads network']['lines gdf'],tunnels_file_path)

In [6]:
def combine_lines_gdf(gdf):
    # 1. Asegurarse de que todas las geometrías son LineString
    def explode_multilines(gdf):
        out_geoms = []
        out_attrs = []
        for idx, row in gdf.iterrows():
            geom = row.geometry
            if isinstance(geom, LineString):
                out_geoms.append(geom)
                out_attrs.append(row.drop("geometry"))
            elif isinstance(geom, MultiLineString):
                for part in geom.geoms:
                    out_geoms.append(part)
                    out_attrs.append(row.drop("geometry"))
        exploded = gpd.GeoDataFrame(out_attrs, geometry=out_geoms, crs=gdf.crs)
        return exploded

    gdf = explode_multilines(gdf)

    # 2. Construir grafo topológico
    g_nx = nx.Graph()
    for i, line in enumerate(gdf.geometry):
        coords = list(line.coords)
        for j in range(len(coords) - 1):
            p1, p2 = coords[j], coords[j + 1]
            g_nx.add_edge(p1, p2, line_id=i)

    # 3. Encontrar cadenas
    def find_chains(g_nx):
        visited_edges = set()
        chains = []

        for u, v in g_nx.edges():
            if (u, v) in visited_edges or (v, u) in visited_edges:
                continue

            chain = [u, v]
            visited_edges.add((u, v))
            visited_edges.add((v, u))

            # expandir hacia u
            current, prev = u, v
            while g_nx.degree[current] == 2:
                neighbors = [n for n in g_nx.neighbors(current) if n != prev]
                if not neighbors:
                    break
                nxt = neighbors[0]
                chain.insert(0, nxt)
                visited_edges.add((current, nxt))
                visited_edges.add((nxt, current))
                prev, current = current, nxt

            # expandir hacia v
            current, prev = v, u
            while g_nx.degree[current] == 2:
                neighbors = [n for n in g_nx.neighbors(current) if n != prev]
                if not neighbors:
                    break
                nxt = neighbors[0]
                chain.append(nxt)
                visited_edges.add((current, nxt))
                visited_edges.add((nxt, current))
                prev, current = current, nxt

            chains.append(chain)

        return chains

    chains = find_chains(g_nx)

    # 4️⃣ Fusionar geometrías y atributos
    merged_records = []
    for chain in chains:
        merged_line = LineString(chain)

        # índices de líneas originales en la cadena
        line_indices = set()
        for j in range(len(chain) - 1):
            edge_data = g_nx.get_edge_data(chain[j], chain[j + 1])
            line_indices.add(edge_data['line_id'])
        sub_gdf = gdf.iloc[list(line_indices)]

        new_record = {}

        # Caso especial: tunnel
        if "tunnel" in gdf.columns:
            if (sub_gdf["tunnel"] == "yes").any(): # type: ignore
                new_record["tunnel"] = "yes"
            else:
                new_record["tunnel"] = "no"

        # Otros atributos: conservar si son constantes
        for col in gdf.columns:
            if col in ("geometry", "tunnel"):
                continue
            values = sub_gdf[col].dropna().unique()
            if len(values) == 1:
                new_record[col] = values[0]
            else:
                new_record[col] = None  # conflicto → se pierde

        new_record["geometry"] = merged_line
        merged_records.append(new_record)

    merged_gdf = gpd.GeoDataFrame(merged_records, crs=gdf.crs)
    return merged_gdf

In [7]:
def gdf_to_nx_buffer_to_lines(lines_gdf,nodes_gdf,buffer_distance):

    # Grafo
    g_nx = nx.Graph()

    # Añadir nodos de estaciones
    for idx, est in nodes_gdf.iterrows():
        g_nx.add_node(idx, **est.to_dict())

    # Diccionario para localizar nodos por coordenadas (tuplas)
    coord_to_node = {tuple(est.geometry.coords[0]): idx for idx, est in nodes_gdf.iterrows()}

    cross_id = 0

    # Diccionario para registrar la mejor vía asignada a cada estación
    station_to_best_via = {}

    # --- Primera pasada: calcular mejor vía para cada estación ---
    for i, via in lines_gdf.iterrows():
        via_buffer = via.geometry.buffer(buffer_distance)

        # Estaciones dentro del buffer
        estaciones_cercanas = nodes_gdf[nodes_gdf.geometry.intersects(via_buffer)].copy()

        if not estaciones_cercanas.empty:
            estaciones_cercanas['distance_to_via'] = estaciones_cercanas.geometry.apply(
                lambda p: via.geometry.distance(p))

            for idx, est in estaciones_cercanas.iterrows():
                dist = est['distance_to_via']
                if (idx not in station_to_best_via) or (dist < station_to_best_via[idx]["dist"]):
                    station_to_best_via[idx] = {"via": i, "dist": dist}

        # --- Crear nodos inicial/final de la vía ---
        line_start = tuple(via.geometry.coords[0])
        if line_start not in coord_to_node:
            start_node = f"start_{cross_id}"
            g_nx.add_node(start_node, geometry=Point(line_start), type="intersection")
            coord_to_node[line_start] = start_node
            cross_id += 1

        line_end = tuple(via.geometry.coords[-1])
        if line_end not in coord_to_node:
            end_node = f"end_{cross_id}"
            g_nx.add_node(end_node, geometry=Point(line_end), type="intersection")
            coord_to_node[line_end] = end_node
            cross_id += 1

    # --- Segunda pasada: asignar vías definitivas y crear aristas ---

    for i, via in lines_gdf.iterrows():
        # Estaciones que quedaron asignadas a esta vía
        estaciones_via = [s for s, best in station_to_best_via.items() if best["via"] == i]

        if len(estaciones_via) > 0:
            subset = nodes_gdf.loc[estaciones_via].copy()
            subset["pos"] = subset.geometry.apply(lambda p: via.geometry.project(p))
            estaciones_ordenadas = subset.sort_values("pos").index.tolist()

            # Conectar estaciones consecutivas
            for j in range(len(estaciones_ordenadas) - 1):
                n1 = estaciones_ordenadas[j]
                n2 = estaciones_ordenadas[j + 1]
                g_nx.add_edge(n1, n2, **via.to_dict())

            # Conectar extremos con estaciones
            line_start = tuple(via.geometry.coords[0])
            line_end = tuple(via.geometry.coords[-1])
            g_nx.add_edge(coord_to_node[line_start], estaciones_ordenadas[0], **via.to_dict())
            g_nx.add_edge(estaciones_ordenadas[-1], coord_to_node[line_end], **via.to_dict())
        else:
            # Sin estaciones asignadas: conectar extremos
            line_start = tuple(via.geometry.coords[0])
            line_end = tuple(via.geometry.coords[-1])
            g_nx.add_edge(coord_to_node[line_start], coord_to_node[line_end], **via.to_dict())

    return g_nx

def gdf_to_nx_no_buffer(lines_gdf, nodes_gdf):

    g_nx = nx.Graph()

    # Añadir nodos de estaciones
    for idx, est in nodes_gdf.iterrows():
        g_nx.add_node(idx, **est.to_dict())

    # Diccionario para localizar nodos por coordenadas (tuplas)
    coord_to_node = {tuple(est.geometry.coords[0]): idx for idx, est in nodes_gdf.iterrows()}

    # Contador para nodos nuevos
    cross_id = 0

    # Procesar cada vía
    for i, via in lines_gdf.iterrows():
        estaciones_conectadas = nodes_gdf[nodes_gdf.intersects(via.geometry)].copy()

        # --- Nodo inicial ---
        line_start = tuple(via.geometry.coords[0])
        if line_start in coord_to_node:
            start_node = coord_to_node[line_start]
        else:
            start_node = f"start_{cross_id}"
            g_nx.add_node(start_node, geometry=Point(line_start), type="intersection")
            coord_to_node[line_start] = start_node
            cross_id += 1

        # --- Nodo final ---
        line_end = tuple(via.geometry.coords[-1])
        if line_end in coord_to_node:
            end_node = coord_to_node[line_end]
        else:
            end_node = f"end_{cross_id}"
            g_nx.add_node(end_node, geometry=Point(line_end), type="intersection")
            coord_to_node[line_end] = end_node
            cross_id += 1

        if len(estaciones_conectadas) >= 1:
            # Ordenar estaciones según la proyección sobre la línea
            estaciones_conectadas["pos"] = estaciones_conectadas.geometry.apply(
                lambda p: via.geometry.project(p)
            )
            estaciones_ordenadas = estaciones_conectadas.sort_values("pos").index.tolist()

            # Conectar estaciones consecutivas
            for j in range(len(estaciones_ordenadas) - 1):
                n1 = estaciones_ordenadas[j]
                n2 = estaciones_ordenadas[j + 1]
                g_nx.add_edge(n1, n2, **via.to_dict())

            # Conectar primera estación con nodo inicial
            g_nx.add_edge(start_node, estaciones_ordenadas[0], **via.to_dict())
            # Conectar última estación con nodo final
            g_nx.add_edge(estaciones_ordenadas[-1], end_node, **via.to_dict())
        else:
            # Si no hay estaciones, conectar directamente inicio y fin
            g_nx.add_edge(start_node, end_node, **via.to_dict())

    return g_nx

def gdf_to_nx_buffer_to_nodes(lines_gdf,nodes_gdf,buffer_distance):

    g_nx= nx.Graph()
    coord_to_node = {}
    cross_id = 0

    boundary = gdf_cut.union_all()
    boundary_line = boundary.boundary  # línea del borde

    def merge_extreme_to_station(g_nx, ext_node, station_node):
        """Conecta vecinos del nodo extremo a la estación y elimina el nodo extremo."""
        neighbors = [n for n in g_nx.neighbors(ext_node) if n != station_node]
        for neighbor in neighbors:
            attr = g_nx.get_edge_data(ext_node, neighbor)
            g_nx.add_edge(station_node, neighbor, **attr)
        g_nx.remove_node(ext_node)

    for i, via in lines_gdf.iterrows():
        line_start = Point(via.geometry.coords[0])
        line_end = Point(via.geometry.coords[-1])

        # Nodo inicio
        if tuple(line_start.coords[0]) in coord_to_node:
            start_node = coord_to_node[tuple(line_start.coords[0])]
        else:
            start_node = f"start_{cross_id}"
            g_nx.add_node(start_node, geometry=line_start, type="intersection")
            coord_to_node[tuple(line_start.coords[0])] = start_node
            cross_id += 1

        # Nodo fin
        if tuple(line_end.coords[0]) in coord_to_node:
            end_node = coord_to_node[tuple(line_end.coords[0])]
        else:
            end_node = f"end_{cross_id}"
            g_nx.add_node(end_node, geometry=line_end, type="intersection")
            coord_to_node[tuple(line_end.coords[0])] = end_node
            cross_id += 1

        # Añadir arista que representa la vía
        g_nx.add_edge(start_node, end_node, **via.to_dict())

    g_nx.remove_edges_from(nx.selfloop_edges(g_nx))

    # 1️⃣ Añadir todas las estaciones al grafo
    for idx, est in nodes_gdf.iterrows():
        g_nx.add_node(idx, **est.to_dict())

    # 2️⃣ Iterar sobre nodos extremos válidos
    for n, data in list(g_nx.nodes(data=True)):
        if data.get("type") != "intersection":
            continue
        if g_nx.degree(n) != 1:
            continue

        # Buscar estaciones cercanas
        candidate_stations = [
            idx for idx, est in nodes_gdf.iterrows()
            if est.geometry.distance(data["geometry"]) <= buffer_distance
        ]

        if candidate_stations:
            # Seleccionar la estación más cercana
            nearest_station = min(
                candidate_stations,
                key=lambda idx: nodes_gdf.loc[idx].geometry.distance(data["geometry"])
            )

            # Conectar nodo extremo con estación
            g_nx.add_edge(nearest_station, n, type="train tracks")

            # Hacer merge: la estación reemplaza al nodo extremo
            merge_extreme_to_station(g_nx, n, nearest_station)

    # Obtener nodos extremos (degree == 1)
    end_nodes = [n for n, d in g_nx.degree() if d == 1]

    # Revisar todos los pares de nodos extremos
    for n1, n2 in itertools.combinations(end_nodes, 2):
        if n1 not in g_nx or n2 not in g_nx:
            continue

        geom1 = g_nx.nodes[n1]["geometry"]
        geom2 = g_nx.nodes[n2]["geometry"]

        # Solo procesar si están dentro de la distancia
        if geom1.distance(geom2) <= buffer_distance:
            type1 = g_nx.nodes[n1].get("type")
            type2 = g_nx.nodes[n2].get("type")

            types=['substations','power sources']

            if type1 in types and type2 in types:
                # Caso especial: ambos son estaciones, solo añadir arista entre ellos
                g_nx.add_edge(n1, n2, type="train tracks")
            else:
                geom = g_nx.nodes[n1]["geometry"]
                if geom.distance(boundary_line)<1e-6:
                    neighbors_n2 = list(g_nx.neighbors(n2))
                    for v in neighbors_n2:
                        if n1 == v:
                            continue
                        g_nx.add_edge(n1, v, type="train tracks")
                    g_nx.remove_node(n2)
                else:
                    # Caso normal: merge/fusión de nodos extremos
                    neighbors_n1 = list(g_nx.neighbors(n1))
                    neighbors_n2 = list(g_nx.neighbors(n2))

                    # Conectar todos los vecinos entre sí (evitando los nodos extremos)
                    for u in neighbors_n1:
                        for v in neighbors_n2:
                            if u == v:
                                continue
                            g_nx.add_edge(u, v, type="train tracks")

                    # Eliminar los dos nodos extremos
                    g_nx.remove_node(n1)
                    g_nx.remove_node(n2)



    # Suponemos que gdf_cut es un polígono (o multipolygon) que define los límites

    isolated_nodes = [n for n, d in g_nx.degree() if d == 0]

    for n in isolated_nodes:
        geom_n = g_nx.nodes[n]["geometry"]

        # Buscar el nodo más cercano que tenga grado > 0
        candidate_nodes = [m for m, d in g_nx.degree() if d > 0]
        if not candidate_nodes:
            continue

        nearest_node = min(
            candidate_nodes,
            key=lambda m: geom_n.distance(g_nx.nodes[m]["geometry"])
        )
        geom_nearest = g_nx.nodes[nearest_node]["geometry"]

        # Línea entre nodo aislado y más cercano
        line = LineString([geom_n, geom_nearest])

        if boundary.contains(line):
            # Caso 1: la línea está dentro -> conectar directamente
            g_nx.add_edge(n, nearest_node, type="train tracks")
        else:
            # Caso 2: la línea se sale -> buscar intersección con el límite
            inter = line.intersection(boundary.boundary)

            if not inter.is_empty:
                # Si hay varias intersecciones, cogemos la más cercana al nodo aislado
                if inter.geom_type == "MultiPoint":
                    inter_point = min(inter.geoms, key=lambda p: geom_n.distance(p))
                else:
                    inter_point = inter

                # Crear un nuevo nodo en el punto de salida
                new_node = f"boundary_{n}"
                g_nx.add_node(new_node, geometry=inter_point, type="boundary")

                # Conectar aislado hasta el límite
                g_nx.add_edge(n, new_node, type="train tracks")

    # Obtener lista de componentes conexas
    components = list(nx.connected_components(g_nx))

    checked = set()  # componentes que ya no se pueden conectar

    while len(components) > 1:
        progress = False  # para saber si en esta iteración conectamos algo

        for smallest_comp in components:
            comp_key = tuple(sorted(map(str, smallest_comp)))
            if comp_key in checked:
                continue

            # Todas las otras componentes
            other_comps = [c for c in components if c != smallest_comp]

            # Inicializar variables para la arista más corta
            min_dist = float("inf")
            closest_pair = None

            # Buscar el par de nodos (uno en smallest_comp, otro en otra componente) más cercano
            for n1 in smallest_comp:
                geom1 = g_nx.nodes[n1]["geometry"]
                for comp in other_comps:
                    for n2 in comp:
                        geom2 = g_nx.nodes[n2]["geometry"]
                        dist = geom1.distance(geom2)
                        if dist < min_dist:
                            min_dist = dist
                            closest_pair = (n1, n2)

            # Conectar si está por debajo del umbral
            if closest_pair and min_dist <= 10*buffer_distance:
                n1, n2 = closest_pair
                g_nx.add_edge(
                    n1, n2, type="train tracks")
                progress = True
                break  # volvemos a recalcular componentes tras una conexión
            else:
                # Marcar esta componente como no conectable en este paso
                checked.add(comp_key)

        if not progress:
            # No hemos podido conectar ninguna componente más
            break

        # Recalcular componentes
        components = list(nx.connected_components(g_nx))
        checked.clear()  # reiniciar porque las componentes han cambiado

    # 3️⃣ Recorrer nodos y comprobar si su geometry toca el borde
    for n, data in g_nx.nodes(data=True):
        geom = data.get("geometry")  # debe ser shapely.Point
        if geom.distance(boundary_line)<1e-6:
            #print(n)
            g_nx.nodes[n]["type"] = "boundary"



    return g_nx

def gdf_to_nx(lines_gdf,nodes_gdf,buffer_distance,buffer_option):

    lines_gdf=combine_lines_gdf(lines_gdf)

    if buffer_distance:
        if buffer_option=='to lines':
            g_nx=gdf_to_nx_buffer_to_lines(lines_gdf,nodes_gdf,buffer_distance)
        elif buffer_option=='to nodes':
            g_nx=gdf_to_nx_buffer_to_nodes(lines_gdf,nodes_gdf,buffer_distance)
        else:
            g_nx=None
    else:
        g_nx = gdf_to_nx_no_buffer(lines_gdf, nodes_gdf)

    if buffer_option != 'to nodes':
        # --- Eliminar componentes conexas de solo 2 o 3 nodos ---
        for comp in list(nx.connected_components(g_nx)):
            if len(comp) == 2 or len(comp)==3:
                g_nx.remove_nodes_from(comp)


    '''# --- Colapsar nodos de intersección ---
    nodes_to_collapse = [n for n, d in g_nx.degree() if d > 2]

    for n in nodes_to_collapse:
        neighbors = list(g_nx.neighbors(n))

        # Conectar todos los vecinos entre sí con MultiGraph (cada conexión será una arista adicional)
        for i in range(len(neighbors)):
            for j in range(i + 1, len(neighbors)):
                g_nx.add_edge(neighbors[i], neighbors[j], via_id=f"collapsed_from_{n}")

        # Eliminar nodo central
        g_nx.remove_node(n)'''

    return g_nx

In [8]:
def nx_to_igraph(network, g_nx):
    # 1️⃣ Lista de nodos y mapeo a índices
    nodes = list(g_nx.nodes())
    node_to_idx = {n: i for i, n in enumerate(nodes)}

    # 2️⃣ Preparar lista de aristas y atributos
    edges = []
    edges_attrs = []

    # Nodos nuevos por túnel
    new_nodes = []
    new_nodes_attrs = {}
    cross_id = 0

    for u, v, attr in g_nx.edges(data=True):
        # Si la arista tiene tunnel="yes", crear nodo intermedio
        if attr.get("tunnel") == "yes":
            #Usar promedio de coordenadas de nodos si existen
            coords_u = g_nx.nodes[u].get("geometry")
            coords_v = g_nx.nodes[v].get("geometry")
            midpoint = Point((coords_u.x + coords_v.x)/2,(coords_u.y + coords_v.y)/2)
            new_node_name = f"tunnel_{cross_id}"
            cross_id += 1

            new_nodes.append(new_node_name)
            new_nodes_attrs[new_node_name] = {"geometry": midpoint, "type": "tunnel"}

            # Partir arista en dos
            edges.append((node_to_idx[u], len(nodes) + len(new_nodes) - 1))
            edges_attrs.append(attr.copy())
            edges.append((len(nodes) + len(new_nodes) - 1, node_to_idx[v]))
            edges_attrs.append(attr.copy())
        else:
            # Arista normal
            edges.append((node_to_idx[u], node_to_idx[v]))
            edges_attrs.append(attr.copy())

    # 3️⃣ Crear grafo vacío
    g_ig = ig.Graph(directed=False)
    g_ig.add_vertices(len(nodes) + len(new_nodes))

    for n, idx in node_to_idx.items():
        g_ig.vs[idx]["name"] = network +' node '+str(n)
        for k, v in g_nx.nodes[n].items():
                g_ig.vs[idx][k] = v

    # 5️⃣ Añadir nodos nuevos (túneles)
    for i, n in enumerate(new_nodes):
        idx = len(nodes) + i
        g_ig.vs[idx]["name"] = network +' node '+str(n)
        for k, v in new_nodes_attrs[n].items():
            g_ig.vs[idx][k] = v

    # 6️⃣ Añadir aristas
    if edges:
        g_ig.add_edges(edges)

    # 7️⃣ Añadir atributos de aristas
    for e_idx, attr in enumerate(edges_attrs):
        for k, v in attr.items():
            g_ig.es[e_idx][k] = v
        # Añadir un nombre a la arista
        src, tgt = g_ig.es[e_idx].tuple  # obtener los nodos de la arista
        g_ig.es[e_idx]["name"] = f"{network} edge {src}-{tgt}"

    return g_ig

In [9]:
for network,dic in networks_dic.items():
    networks_dic[network]['igraph']=nx_to_igraph(network,gdf_to_nx(
        networks_dic[network]['lines gdf'],
        networks_dic[network]['nodes gdf'],
        networks_dic[network]['buffer distance'],networks_dic[network]['buffer option']))

    dic['igraph'].vs["network"] = [network] * dic['igraph'].vcount()
    dic['igraph'].es["network"] = [network] * dic['igraph'].ecount()

PLOTS

In [ ]:
def plot_gdf(network,gdf_lines,gdf_nodes,gdf_cut):
    fig, ax = plt.subplots(figsize=(10, 8))
    gdf_cut.plot(ax=ax, color="lightgrey", edgecolor="black", figsize=(8, 8))
    gdf_lines.plot(ax=ax, color="green", edgecolor="black", linewidth=0.5)
    gdf_nodes.plot(ax=ax, color="red", markersize=0.5)

    plt.title("Mapa de "+network)
    file_name="mapa "+network+".png"
    plt.savefig(file_name, dpi=300, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)

def plot_ig_graph(network, g_ig, gdf_cut, node_colors,edge_colors):
    # Extraer coordenadas de los nodos
    x_coords = [v['geometry'].x for v in g_ig.vs]
    y_coords = [v['geometry'].y for v in g_ig.vs]
    coords_layout = list(zip(x_coords, y_coords))

    fig, ax = plt.subplots(figsize=(10, 8))

    # Dibujar capa de cortes si existe
    if gdf_cut is not None:
        gdf_cut.plot(ax=ax, facecolor="none", edgecolor="black")

    ig.plot(
        g_ig,
        target=ax,
        layout=coords_layout,  # <-- usamos las coordenadas reales
        vertex_size=20,
        vertex_color=node_colors,
        edge_color=edge_colors,
        vertex_frame_width=3,
        edge_width=1,
        vertex_label=None,
        margin=0
    )

    # Ajustar límites
    if gdf_cut is not None:
        xmin, ymin, xmax, ymax = gdf_cut.total_bounds
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
    plt.title("Grafo geométrico de "+network)

    # Guardar y abrir
    file_name ="grafo geom "+network+".png"
    plt.savefig(file_name, dpi=1200, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)

network='Energy network'
plot_gdf(network,networks_dic[network]['lines gdf'],networks_dic[network]['nodes gdf'],gdf_cut)
plot_ig_graph(network, networks_dic[network]['igraph'], gdf_cut, 'red','blue')

###############################

In [10]:
def intercon_igraph(interconnection,g_combined,intercon_dic):

    network_1, network_2 = interconnection.split(" - ")

    if intercon_dic['connected elements'][0]=='all':
        target_nodes_1 = [v for v in g_combined.vs if v['network'] == network_1]
    else:
        target_nodes_1 = [v for v in g_combined.vs if v['network'] == network_1 and v['type']==intercon_dic['connected elements'][0]]

    if intercon_dic['connected elements'][1]=='all':
        target_nodes_2 = [v for v in g_combined.vs if v['network'] == network_2]
    else:
        target_nodes_2 = [v for v in g_combined.vs if v['network'] == network_2 and v['type']==intercon_dic['connected elements'][1]]

        # 2️⃣ Iterar sobre pares y conectar si cumplen distancia
    if intercon_dic['method']=='all in buffer distance':
        for v1 in target_nodes_1:
            geom1 = v1['geometry']
            for v2 in target_nodes_2:
                geom2 = v2['geometry']
                if geom1.distance(geom2) <= intercon_dic['buffer distance']:
                    # Añadir arista entre los nodos
                    g_combined.add_edge(v1.index, v2.index, network=interconnection ,type='interconnection',name=f"{interconnection} edge {v1.index}-{v2.index}")

    elif intercon_dic['method']=='closest':
        # Iterar sobre todos los nodos de net1
        for v1 in target_nodes_1:
            geom1 = v1["geometry"]

            # Buscar el nodo más cercano en net2
            nearest_node = min(
                target_nodes_2,
                key=lambda v2: geom1.distance(v2["geometry"])
            )

            # Añadir arista al grafo con tipo y geometry
            g_combined.add_edge(v1.index, nearest_node.index, network=interconnection ,type="interconnection",name=f"{interconnection} edge {v1.index}-{nearest_node.index}")


    return g_combined

def networks_intercon(networks_dic,interconections_dic):

    allowed_types=['tunnel']
    for dic in networks_dic.values():
        for node_keys in dic['nodes file paths'].keys():
            allowed_types.append(node_keys)

    for interconection, intercon_dic in  interconections_dic.items():

        network_1, network_2 = interconection.split(" - ")

        # Crear un grafo vacío
        g_combined = ig.Graph(directed=False)

        # Añadir los nodos de g1
        for v in networks_dic[network_1]['igraph'].vs:
            if v["type"] in allowed_types:
                g_combined.add_vertex(**v.attributes())
        # Añadir los nodos de g2
        for v in networks_dic[network_2]['igraph'].vs:
            if v["type"] in allowed_types:
                g_combined.add_vertex(**v.attributes())

        intercon_dic['igraph']=intercon_igraph(interconection,g_combined,intercon_dic)

In [11]:
networks_intercon(networks_dic,interconections_dic)

PLOTS

In [ ]:
def plot_ig_graph(network, g_ig, gdf_cut, node_colors,edge_colors):
    # Extraer coordenadas de los nodos
    x_coords = [v['geometry'].x for v in g_ig.vs]
    y_coords = [v['geometry'].y for v in g_ig.vs]
    coords_layout = list(zip(x_coords, y_coords))

    fig, ax = plt.subplots(figsize=(10, 8))

    # Dibujar capa de cortes si existe
    if gdf_cut is not None:
        gdf_cut.plot(ax=ax, facecolor="none", edgecolor="black")

    ig.plot(
        g_ig,
        target=ax,
        layout=coords_layout,  # <-- usamos las coordenadas reales
        vertex_size=20,
        vertex_color=node_colors,
        edge_color=edge_colors,
        vertex_frame_width=3,
        edge_width=1,
        vertex_label=None,
        margin=20
    )

    # Ajustar límites
    if gdf_cut is not None:
        xmin, ymin, xmax, ymax = gdf_cut.total_bounds
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
    plt.title("Grafo geométrico de "+network)

    # Guardar y abrir
    file_name ="grafo geom "+network+".png"
    plt.savefig(file_name, dpi=1200, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)

for network, dic in networks_dic.items():
        network_nodes_types=[key for key in dic['nodes file paths'].keys()]
        node_colors = ['red' if v['type'] in network_nodes_types else 'yellow' for v in networks_dic[network]['igraph'].vs]
        plot_ig_graph(network, networks_dic[network]['igraph'], gdf_cut, node_colors,'blue')


for networks,dic in interconections_dic.items():
    network_1= networks.split(" - ")[0].strip()
    node_colors = ['red' if v['network']==network_1 else 'yellow' for v in dic['igraph'].vs]
    plot_ig_graph(networks,dic['igraph'],gdf_cut,node_colors,'blue')

###############################

In [12]:
def assign_users_and_flux_transport(g_ig):

    # -------------------------------
    # Usuarios por nodo
    # -------------------------------
    g_ig.vs["users"] = [random.randint(50, 200) for _ in g_ig.vs]

    # -------------------------------
    # Inicializar flujos de salida por aristas
    # -------------------------------

    shares_by_node = {v["name"]: {} for v in g_ig.vs}

    # Asignar porcentajes de salida a las aristas (sum ≤ 1)
    for v in g_ig.vs:
        neighbors = g_ig.neighbors(v, mode="ALL")
        if not neighbors:
            continue

        weights = [random.random() for _ in neighbors]
        total = sum(weights)
        factor = random.uniform(0.5, 0.9)  # suma de salidas ≤1
        shares = [round((w / total) * factor, 2) for w in weights]

        for n, s in zip(neighbors, shares):
            shares_by_node[v["name"]][g_ig.vs[n]["name"]] = s

    # Asignar atributos a las aristas
    for e in g_ig.es:
        u_name, v_name = g_ig.vs[e.source]["name"], g_ig.vs[e.target]["name"]
        e[u_name] = shares_by_node[u_name].get(v_name)
        e[v_name] = shares_by_node[v_name].get(u_name)

    # -------------------------------
    # Calcular flujo de llegada
    # -------------------------------

    for v in g_ig.vs:
        name = v["name"]
        incoming_flows = []
        total_incoming = 0

        # Recoger todos los flujos entrantes
        for e in g_ig.es:
            u_name, w_name = g_ig.vs[e.source]["name"], g_ig.vs[e.target]["name"]
            if w_name == name:  # flujo desde u hacia v
                flow_value = e[u_name] * g_ig.vs[e.source]["users"]
                incoming_flows.append((e, "source", flow_value))
                total_incoming += flow_value
            elif u_name == name:  # flujo desde w hacia v
                flow_value = e[w_name] * g_ig.vs[e.target]["users"]
                incoming_flows.append((e, "target", flow_value))
                total_incoming += flow_value

        # Escalar flujos si exceden users
        if total_incoming > v["users"]:
            factor = v["users"] / total_incoming
            for e, attr, _ in incoming_flows:
                u_name, v_name = g_ig.vs[e.source]["name"], g_ig.vs[e.target]["name"]
                if attr == "source":
                    e[u_name] *= factor
                else:
                    e[v_name] *= factor

def assign_users_and_flux_energy(g_ig_energy,gdf_energy_nodes,cities_file_path):
    import warnings
    from collections import Counter

    warnings.filterwarnings("ignore", message=".*Geometry is in a geographic CRS.*")

    gdf_cities = gpd.read_file(cities_file_path)
    gdf_cities = gdf_cities.to_crs(gdf_energy_nodes.crs)

    # --- 0. Filtrar subestaciones ---
    gdf_substations_sel = gdf_energy_nodes[gdf_energy_nodes["type"] == "substations"]

    # --- 1. Subestación -> ciudad más cercana ---
    sub_to_city = {}
    for i, sub_geom in gdf_substations_sel.geometry.items():
        dists = gdf_cities.geometry.distance(sub_geom)
        idx_nearest = dists.idxmin()
        sub_to_city[i] = idx_nearest

    # --- 2. Contar cuántas subestaciones por ciudad ---
    city_counts = Counter(sub_to_city.values())

    # --- 3. Calcular PPL_per_sub en un diccionario auxiliar ---
    city_to_ppl = {}
    for idx, row in gdf_cities.iterrows():
        n_subs = city_counts.get(idx, 0)
        if n_subs > 0:
            city_to_ppl[idx] = row["PPL"] / n_subs
        else:
            city_to_ppl[idx] = None

    # --- 4. Construir diccionario final solo con esas subestaciones ---
    sub_to_ppl_dict = {}
    for sub_idx, city_idx in sub_to_city.items():
        sub_geom = gdf_substations_sel.geometry.loc[sub_idx]
        ppl_value = city_to_ppl[city_idx]
        sub_to_ppl_dict[sub_geom] = ppl_value

    # --- Añadir atributo "users" a todos los nodos ---
    users_vals = []
    for v in g_ig_energy.vs:
        geom = v["geometry"]
        if geom in sub_to_ppl_dict:
            users_vals.append(sub_to_ppl_dict[geom])
        else:
            users_vals.append(0)

    g_ig_energy.vs["users"] = [round(u) for u in users_vals]
    g_ig_energy.vs["energy"]=1
    g_ig_energy.es["energy"]=1

def assign_users_and_flux_energy_intercons(interconections_dic,networks_dic):

    for e in interconections_dic['Railway network - Energy network']['igraph'].es:
        u= interconections_dic['Railway network - Energy network']['igraph'].vs[e.source]
        v= interconections_dic['Railway network - Energy network']['igraph'].vs[e.target]

        geom_u = u["geometry"]
        geoms_ig_energy_set = set(networks_dic['Energy network']['igraph'].vs["geometry"])
        if geom_u in geoms_ig_energy_set:
            e[u['name']]=1
        else:
            e[v['name']]=1

    for e in interconections_dic['Roads network - Energy network']['igraph'].es:
        u= interconections_dic['Roads network - Energy network']['igraph'].vs[e.source]
        v= interconections_dic['Roads network - Energy network']['igraph'].vs[e.target]

        geom_u = u["geometry"]
        geoms_ig_energy_set = set(networks_dic['Energy network']['igraph'].vs["geometry"])
        if geom_u in geoms_ig_energy_set:
            e[u['name']]=1
        else:
            e[v['name']]=1

In [13]:
transport_igraphs_list=[networks_dic['Roads network']["igraph"],networks_dic['Railway network']["igraph"],interconections_dic['Roads network - Railway network']["igraph"]]
transport_graph=ig.union(transport_igraphs_list,byname=True)
assign_users_and_flux_transport(transport_graph)

In [14]:
assign_users_and_flux_energy(networks_dic['Energy network']["igraph"],networks_dic['Energy network']["nodes gdf"],cities_file_path)
assign_users_and_flux_energy_intercons(interconections_dic,networks_dic)

TESTING

In [ ]:
import pandas as pd
g=networks_dic['Energy network']["igraph"]

df_nodos = pd.DataFrame({
    "name": g.vs["name"],
    "type": g.vs["type"],
    "users": g.vs["users"]
})

print(df_nodos.head())

###############################


In [15]:
all_graphs_list=[interconections_dic['Roads network - Energy network']["igraph"],interconections_dic['Railway network - Energy network']["igraph"],transport_graph,networks_dic['Energy network']["igraph"]]
main_graph=ig.union(all_graphs_list,byname=True)

TESTING

In [ ]:
import pandas as pd
from collections import Counter

def summarize_vertex_attrs(g):
    summary = {}

    for attr in g.vs.attributes():  # lista de atributos de vértices
        values = g.vs[attr]         # todos los valores de ese atributo
        counter = Counter(values)   # cuenta las ocurrencias
        summary[attr] = {
            "unique_values": list(counter.keys()),
            "counts": dict(counter)
        }

    return summary

# Ejemplo de uso:
summary = summarize_vertex_attrs(main_graph)

# Si quieres verlo en tabla (pandas)
rows = []
for attr, info in summary.items():
    for val, count in info["counts"].items():
        rows.append({"attribute": attr, "value": val, "count": count})

df_summary = pd.DataFrame(rows)
print(df_summary)

###############################


In [50]:
def resil_vun_analysis(combined_graph,t_0,n,params_dic, dt, fail_drop):

    scenarios_dic = {}

    if n==1:

        '''v_f=combined_graph.vs.find(name= 'Energy network 34999')
        print('Failed asset ' + str(0) + '/' + str(combined_graph.vcount()) + ': node ' + v_f["name"])
        scenarios_dic[v_f["name"]] = simulate_scenario([v_f], t_0, combined_graph, params_dic, dt, fail_drop)
        # plot_failure_scen(v_f["name"],scenarios_dic)'''

        print('Nodos: ' + str(combined_graph.vcount()))
        i = 1
        for v_f in combined_graph.vs:
            #if v_f['network']=='Energy network':
                #v_f=combined_graph.vs.find(name= 'Energy network node end_182')
            print('Failed asset ' + str(i) + '/' + str(combined_graph.vcount()) + ': node ' + v_f["name"])
            scenarios_dic[v_f["name"]] = simulate_scenario([v_f], t_0, combined_graph, params_dic, dt, fail_drop)
            # plot_failure_scen(v_f["name"],scenarios_dic)
            i += 1

        print('Aristas: ' + str(combined_graph.ecount()))
        i = 1
        for e_f in combined_graph.es:
            print('Failed asset ' + str(i) + '/' + str(combined_graph.ecount()) + ': edge ' + e_f["name"])
            scenarios_dic[e_f["name"]] = simulate_scenario([e_f], t_0, combined_graph, params_dic, dt, fail_drop)
            # plot_failure_scen(e_f.index,scenarios_dic)
            i += 1

    return scenarios_dic

def simulate_scenario(a_f_list,t_0,g_ig,params_dic, dt, fail_drop):
    scenario_dic={}
    total_users=0
    for v in g_ig.vs:
        scenario_dic[v['name']]=[v['users']]
        total_users+=v['users']
    #print(total_users)
    state_flag='initial'
    t=dt

    g_ig_energy=g_ig.subgraph([v.index for v in g_ig.vs if v["network"] == 'Energy network'])
    boundary_nodes = [v['name'] for v in g_ig_energy.vs if v["type"] == "boundary"]
    power_source_nodes = [v['name'] for v in g_ig_energy.vs if v["type"] == "power sources"]
    generator_nodes = list(set(boundary_nodes + power_source_nodes))
    v_ener_name_to_idx = {v["name"]: v.index for v in g_ig_energy.vs if "name" in v.attributes()}
    e_ener_name_to_idx = {e["name"]: e.index for e in g_ig_energy.es if "name" in e.attributes()}

    g_ig_failed_elements_energy=g_ig_energy.copy()
    comps_energy = g_ig_failed_elements_energy.components(mode="weak")
    membership = comps_energy.membership
    comp_sets = [set(c) for c in comps_energy]
    all_names = np.array(g_ig_failed_elements_energy.vs["name"])
    v_name_to_idx = {v["name"]: v.index for v in g_ig_failed_elements_energy.vs if "name" in v.attributes()}

    while state_flag!='finished':
        actual_users=0
        t_ref = np.maximum(dt, round(t - params_dic['tFa'],1))
        index = int(round(t_ref / dt)) - 1

        # 1. Calcular fallos (sin copiar grafo)
        change_failed_v = False
        change_failed_e = False
        for element in a_f_list:
            if element['network'] == 'Energy network':
                if isinstance(element, ig.Vertex):
                    capacity_profile=asset_failure_profile(t,1,t_0,1,params_dic)
                    energy_state=element["energy"]
                    if capacity_profile < 1 and energy_state==1:
                        state_flag = "failure"
                        change_failed_v=True
                        element["energy"] = 0
                        g_ig_energy.vs[v_ener_name_to_idx[element["name"]]]["energy"]=0
                    elif capacity_profile== 1 and energy_state==0:
                        element["energy"] = 1
                        g_ig_energy.vs[v_ener_name_to_idx[element["name"]]]["energy"]=1
                        change_failed_v=True
                elif isinstance(element, ig.Edge):
                    flux = asset_failure_profile(t, 1, t_0, 1, params_dic)
                    energy_state=element["energy"]
                    if flux < 1 and energy_state==1:
                        state_flag = "failure"
                        change_failed_e=True
                        element["energy"] = 0
                        g_ig_energy.es[e_ener_name_to_idx[element["name"]]]["energy"]=0
                    elif flux== 1 and energy_state==0:
                        element["energy"] = 1
                        g_ig_energy.es[e_ener_name_to_idx[element["name"]]]["energy"]=1
                        change_failed_e=True



        # 3. Solo si hubo fallos → recomputar componentes energía
        if change_failed_v or change_failed_e:
            active_nodes = [v.index for v in g_ig_energy.vs if v["energy"] == 1]
            active_edges = [e.index for e in g_ig_energy.es if e["energy"] == 1]
            g_ig_failed_elements_energy = g_ig_energy.subgraph_edges(active_edges, delete_vertices=False).induced_subgraph(active_nodes)

            comps_energy = g_ig_failed_elements_energy.components(mode="weak")
            membership = comps_energy.membership
            comp_sets = [set(c) for c in comps_energy]
            all_names = np.array(g_ig_failed_elements_energy.vs["name"])
            v_name_to_idx = {v["name"]: v.index for v in g_ig_failed_elements_energy.vs if "name" in v.attributes()}

        for v in g_ig.vs:
            if v['network']=='Energy network':

                if v['energy']==1:

                    # Obtener índice del nodo en g_ig_energy
                    node_idx = v_name_to_idx[v["name"]]

                    # ID del componente al que pertenece
                    comp_id = membership[node_idx]

                    # Conjunto de nodos activos en ese componente
                    reachable_nodes = comp_sets[comp_id]
                    reachable_names = all_names[list(reachable_nodes)]

                    if set(generator_nodes) & set(reachable_names):
                      node_users=v['users']
                    else:
                       #print(v['name'])
                       node_users=0
                else:
                    node_users=0
                scenario_dic[v['name']].append(node_users)

                actual_users+=node_users

            else:
                neighbors_list = g_ig.vs[g_ig.neighbors(v)]
                node_users = v['users']
                power=1
                for u in neighbors_list:
                    e=g_ig.es[g_ig.get_eid(u.index, v.index)]
                    if u['network']=='Energy network':
                        if e in a_f_list:
                            flux_e=asset_failure_profile(t,e[u['name']],t_0,1,params_dic)
                            if flux_e<1:
                                state_flag='failure'
                        else:
                            flux_e=e[u['name']]
                        power*=u['energy']*flux_e
                        #if power==0:
                            #print(u['energy'],flux_e)

                    else:
                        if e in a_f_list:
                            flux_e=asset_failure_profile(t,e[u['name']],t_0,fail_drop,params_dic)
                            #print(e[u['name']])
                            #print(flux_e)
                            if flux_e<e[u['name']]:
                                state_flag='failure'
                        else:
                            flux_e=e[u['name']]
                        reference_value=scenario_dic[u['name']][index]
                        node_users=node_users-(u['users']*e[u['name']] - reference_value*flux_e)
                if power==0:
                    node_users=0
                elif power==1:
                    if v in a_f_list:
                        capacity_profile=asset_failure_profile(t,v['users'],t_0,fail_drop,params_dic)
                        if capacity_profile<v['users']:
                            state_flag='failure'
                        if node_users>capacity_profile:
                            node_users=capacity_profile
                #if node_users<v['users']:
                    #print(power)
                    #print(v['name'])
                    #print(v['users'])
                    #print(node_users)
                scenario_dic[v['name']].append(node_users)
                actual_users+=node_users

        if state_flag == 'failure' and abs(actual_users - total_users) < 0.1:
            state_flag='finished'
        #print(actual_users,t,state_flag)
        t=round(t+dt,1)
    return scenario_dic

def asset_failure_profile(t,initial_value,t_f,fail_drop,params_dic):
    if t<t_f+params_dic['tFa']:
        return initial_value
    elif t_f+params_dic['tFa']<=t<t_f+params_dic['tFa']+params_dic['Rc0']+params_dic['tRc']:
        return initial_value*(1-fail_drop)
    else:
        return initial_value

In [49]:
fail_drop = 1.0  # Total
t_0 = 1.0
params_dic = {'tFa': 1.0, 'Rc0': 1.0, 'tRc': 4.0}
dt = 0.1  # (h)
n=1

main_graph.vs["energy"] = [1] * main_graph.vcount()
main_graph.es["energy"] = [1] * main_graph.ecount()

scenarios_dic1=resil_vun_analysis(main_graph,t_0,n,params_dic, dt, fail_drop)

print('Analysis finished')

Nodos: 897
Failed asset 1/897: node Energy network node 258
Failed asset 2/897: node Roads network node end_6
Failed asset 3/897: node Roads network node end_251


KeyboardInterrupt: 

In [ ]:
with open("datos.json", "w", encoding="utf-8") as f:
    json.dump(scenarios_dic1, f, ensure_ascii=False, indent=4)

PLOTS y TESTING

In [ ]:
def node_attrs_by_name(g, node_name):
    v = g.vs.find(name=node_name)   # buscar el nodo por name
    return {attr: v[attr] for attr in g.vs.attributes()}


# Ejemplo de uso:
node_info = node_attrs_by_name(main_graph, "Energy network node 63")
for k, v in node_info.items():
    print(f"{k}: {v}")

In [ ]:
def plot_ig_graph(network, g_ig, gdf_cut, node_colors,edge_colors):
    # Extraer coordenadas de los nodos
    x_coords = [v['geometry'].x for v in g_ig.vs]
    y_coords = [v['geometry'].y for v in g_ig.vs]
    coords_layout = list(zip(x_coords, y_coords))

    fig, ax = plt.subplots(figsize=(10, 8))

    # Dibujar capa de cortes si existe
    #if gdf_cut is not None:
        #gdf_cut.plot(ax=ax, facecolor="none", edgecolor="black")

    ig.plot(
        g_ig,
        target=ax,
        layout=coords_layout,  # <-- usamos las coordenadas reales
        vertex_size=20,
        vertex_color=node_colors,
        edge_color=edge_colors,
        vertex_frame_width=0,
        edge_width=0.1,
        vertex_label=None,
        margin=20
    )

    # Ajustar límites
    if gdf_cut is not None:
        xmin, ymin, xmax, ymax = gdf_cut.total_bounds
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
    plt.title("Grafo geométrico de "+network)

    # Guardar y abrir
    file_name ="grafo geom "+network+".png"
    plt.savefig(file_name, dpi=1200, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)

network="Energy network"
g_ig=networks_dic[network]['igraph']

#boundary_nodes = [v.index for v in g_ig.vs if v["type"] == "boundary"]
#power_source_nodes = [v.index for v in g_ig.vs if v["type"] == "power sources"]
#generator_nodes = list(set(boundary_nodes + power_source_nodes))
node_colors = ['red' if v.index in  generator_nodes else 'yellow' for v in g_ig.vs]

red_nodes_list=['Energy network node 219','Energy network node end_167',
'Energy network node end_173',
'Energy network node 147',
'Energy network node 143',
'Energy network node boundary_63',
'Energy network node 63']
node_colors = ['red' if v['name'] in  red_nodes_list else 'yellow' for v in g_ig.vs]

plot_ig_graph("pruebas caminos",g_ig,gdf_cut,node_colors,'blue')

In [ ]:
for network, dic in networks_dic.items():
        #plot_gdf(network,networks_dic[network]['lines gdf'],networks_dic[network]['nodes gdf'],gdf_cut)
        node_colors = ['red' if v['type'] == 'station' else 'yellow' for v in networks_dic[network]['igraph'].vs]
        #plot_ig_graph(network, networks_dic[network]['igraph'], gdf_cut, node_colors,'blue')
        layout = networks_dic[network]['igraph'].layout("fr")  # "fr" = Fruchterman-Reingold, similar a nx.spring_layout
        plot_ig_asbtract(network,layout,networks_dic[network]['igraph'],node_colors,'blue')

In [ ]:
def plot_ig_users_flux(g_ig, gdf_cut, node_colors,edge_colors):
    """
    Dibuja un grafo no dirigido mostrando flechas simuladas desde cada nodo
    para indicar el flujo que sale por cada arista.
    Cada arista tiene dos atributos: flujo desde cada nodo.
    """

    # Extraer coordenadas
    x_coords = [v['geometry'].x for v in g_ig.vs]
    y_coords = [v['geometry'].y for v in g_ig.vs]
    coords_layout = list(zip(x_coords, y_coords))
    '''layout = g_ig.layout("kk")
    coords_layout = [(x, y) for x, y in layout.coords]
    x_coords = [c[0] for c in coords_layout]
    y_coords = [c[1] for c in coords_layout]'''

    # Creamos listas para flechas y etiquetas
    arrows = []
    edge_labels = []
    edge_widths = []

    for e in g_ig.es:
        u_idx, v_idx = e.source, e.target
        u_name = str(g_ig.vs[u_idx]["name"])
        v_name = str(g_ig.vs[v_idx]["name"])

        # Flujo desde cada nodo
        flow_u = e[u_name]
        flow_v = e[v_name]

        # Simulamos flecha desde u a v
        arrows.append(((coords_layout[u_idx], coords_layout[v_idx]), flow_u))
        # Simulamos flecha desde v a u
        arrows.append(((coords_layout[v_idx], coords_layout[u_idx]), flow_v))

    # Plot con matplotlib
    fig, ax = plt.subplots(figsize=(12,10))
    gdf_cut.plot(ax=ax, facecolor="none", edgecolor="black")
    ax.set_aspect('equal')
    ax.set_title("Flujos de nodos")

    # Dibujar nodos
    ax.scatter(x_coords, y_coords, s=5, c='red', zorder=3)
    for i, v in enumerate(g_ig.vs):
        ax.text(
            x_coords[i],
            y_coords[i],
             f"{v['name']}\n({v['users']})",  # Texto: nombre (usuarios)
            fontsize=1,
            color=node_colors,        # Contrasta mejor con el nodo rojo
            ha="center",          # Centrado horizontal
            va="center",          # Centrado vertical
            zorder=4,
        )


    # Dibujar flechas y etiquetas
    for (start, end), flow in arrows:
        # Dibujar la flecha
        ax.annotate(
            "", xy=end, xytext=start,
            arrowprops=dict(arrowstyle="->", color=edge_colors, lw=0.8),
            zorder=2
        )

        # Vector de la arista
        dx = end[0] - start[0]
        dy = end[1] - start[1]
        length = (dx**2 + dy**2)**0.5

        # Posición del texto: cerca del inicio, a un 15% de la arista
        t = 0.15
        x_text = start[0] + t * dx
        y_text = start[1] + t * dy

        # Desplazamiento perpendicular opcional para evitar solapamientos
        perp_offset = 0.02 * length
        x_text += -perp_offset * dy / length
        y_text += perp_offset * dx / length

        # Dibujar la etiqueta
        ax.text(x_text, y_text, f"{flow:.2f}",
                fontsize=1, color='black', zorder=4,
                ha='center', va='center')

    xmin, ymin, xmax, ymax = gdf_cut.total_bounds
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    plt.title("Grafo de usuarios y flujos")

    file_name ="grafo_ig_users_flux.png"
    plt.savefig(file_name, dpi=1200, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)

In [ ]:
node_colors = ['red' if v['network']=='Roads network'  else 'yellow' for v in combined_graph.vs]
#plot_ig_graph("combinado", combined_graph, gdf_cut, node_colors,'blue')

'''nodes_A = [v.index for v in combined_graph.vs if v["network"] == 'Roads network']
nodes_B = [v.index for v in combined_graph.vs if v["network"] == 'Railway network']
sub_A = combined_graph.subgraph(nodes_A)
sub_B = combined_graph.subgraph(nodes_B)
# --- 2️⃣ Layouts internos (FR)
layout_A = np.array(sub_A.layout("fr"))
layout_B = np.array(sub_B.layout("fr"))
# Normalizar tamaños
if layout_A.shape[0] > 0:
    layout_A = layout_A / np.max(np.abs(layout_A)) * 5
if layout_B.shape[0] > 0:
    layout_B = layout_B / np.max(np.abs(layout_B)) * 5
# --- 3️⃣ Trasladar bloques
layout_A[:, 0] -= 10   # A hacia la izquierda
layout_B[:, 0] += 10   # B hacia la derecha
# --- 4️⃣ Combinar layouts
layout = np.zeros((combined_graph.vcount(), 2))
for i, node in enumerate(nodes_A):
    layout[node] = layout_A[i]
for i, node in enumerate(nodes_B):
    layout[node] = layout_B[i]
layout=layout.tolist()
plot_ig_asbtract("combinado",layout,combined_graph,node_colors,'blue')'''

plot_ig_users_flux(combined_graph, gdf_cut, 'black','blue')

In [ ]:
def plots_resil_vun_analysis(scenarios_dic):

    plt.figure(figsize=(8, 5))
    for node, dic in scenarios_dic.items():
        lista = [sum(x) for x in zip(*dic.values())]
        x = [i * 0.1 for i in range(len(lista))]
        plt.plot(x, lista, marker='.', markersize=1, label=node)  # gráfica de línea con puntos
        # plt.text(x[-1] + 0.1, lista[-1], node, va="center")
    plt.title("Network performance curves with 1 failed asset at t=1h with 100% drop")
    plt.ylabel("Network performance (nº users)")
    plt.xlabel("t (h)")
    plt.grid(True)
    file_name ="performance curves all scens.png"
    plt.savefig(file_name, dpi=1200, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)

    '''failed_asset = list(scenarios_dic.keys())[0]
    #failed_asset = "Energy network 35169"
    plt.figure(figsize=(8, 5))
    lista = [sum(x) for x in zip(*scenarios_dic[failed_asset].values())]
    x = [i * 0.1 for i in range(len(lista))]
    plt.plot(x, lista, marker='.', markersize=1)  # gráfica de línea con puntos
    plt.title("Network performance curve with 1 failed asset at t=1h with 100% drop. Failed asset: "+failed_asset)
    plt.ylabel("Network performance (nº users)")
    plt.xlabel("t (h)")
    plt.grid(True)
    file_name ="performance curve failed "+failed_asset+".png"
    plt.savefig(file_name, dpi=1200, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)

    plt.figure(figsize=(8, 5))
    for node, lista in scenarios_dic[failed_asset].items():
        # Solo plotear si hay algún cambio en la lista
        if max(lista) - lista[0] > 1 or lista[0] - min(lista) > 1:
            x = [i * 0.1 for i in range(len(lista))]
            plt.plot(x, lista, marker='.', markersize=1, label=node)
    plt.title(failed_asset)
    plt.title("Node performance curve with 1 failed asset at t=1h with 100% drop. Failed asset: " + failed_asset)
    plt.ylabel("Node performance (nº users)")
    plt.xlabel("t (h)")
    plt.grid(True)
    plt.legend()
    file_name ="nodes performance failed "+failed_asset+".png"
    plt.savefig(file_name, dpi=1200, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)'''

In [ ]:
plots_resil_vun_analysis(scenarios_dic1)

In [ ]:
for failed_asset, dic in scenarios_dic1.items():

    plt.figure(figsize=(8, 5))
    for node, lista in dic.items():
        # Solo plotear si hay algún cambio en la lista
        if max(lista) - lista[0] > 1 or lista[0] - min(lista) > 1:
            x = [i * 0.1 for i in range(len(lista))]
            plt.plot(x, lista, marker='.', markersize=1, label=node)
    plt.title(failed_asset+' // Type: '+main_graph.vs.find(name=failed_asset)['type'])
    plt.xlabel("t (h)")
    plt.ylabel("Network performance (nº users)")
    plt.grid(True)
    plt.legend()
    plt.show()
    '''file_name ="performance curve failed "+failed_asset+".png"
    plt.savefig(file_name, dpi=1200, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)'''

In [ ]:
for network,dic in networks_dic.items():
    print("Atributos de nodos de "+network+":", dic['igraph'].vs.attributes())
    print("Atributos de aristas de "+network+":", dic['igraph'].es.attributes())
for networks in networks_intercon_dic.keys():
    print("Atributos de nodos de "+networks+":", networks_intercon_dic[networks].vs.attributes())
    print("Atributos de aristas de "+networks+":", networks_intercon_dic[networks].es.attributes())

In [ ]:
print("Atributos de nodos de main:", combined_graph.vs.attributes())
print("Atributos de aristas de main:", combined_graph.es.attributes())

In [ ]:
print([v["name"] for v in [v for v in networks_dic['Energy network']['igraph'].vs if v["type"] == "boundary"]])

In [ ]:
for v in networks_dic['Energy network']['igraph'].vs:
    print(v["name"],v['users'])